In [6]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
!nvidia-smi

Wed Apr  3 07:12:27 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.161.07             Driver Version: 535.161.07   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          On  | 00000000:07:00.0 Off |                    0 |
| N/A   49C    P0             305W / 400W |  12822MiB / 40960MiB |     77%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

### To reproduce this experiment, you must clone below repository fastText to get the embedding

In [7]:
#git clone https://github.com/facebookresearch/fastText.git
%cd fastText


[Errno 2] No such file or directory: 'fastText'
/workspace/Tugas Akhir/fastText


In [8]:
!pip install fasttext

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [9]:
import fasttext
import fasttext.util
#fasttext.util.download_model('id', if_exists='ignore')  # English
# to get reduced model below without `fasttext.util.reduce_model(ft, 200)`, you can download through this link
# https://drive.google.com/file/d/1n8ykA9v20lmDwTuTY0wp72vDaVosh1d4/view?usp=sharing
ft = fasttext.load_model('ft_200dim.bin')
ft.get_dimension()


200

In [8]:
# fasttext.util.reduce_model(ft, 200)
# ft.get_dimension()


In [9]:
# ft.save_model("ft_200dim.bin")

In [10]:
ft.get_word_vector('hello').shape

(200,)

In [11]:
def get_vect(token):
    try:
        return ft.get_word_vector(token)
    except:
        try:
            return ft.get_word_vector(token)
        except:
            return ft.get_word_vector("hello")-ft.get_word_vector("hello") # return zero vector an oov word

In [12]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [13]:
from nltk.tokenize import word_tokenize
def get_vector_for_sentence(sentence, max_num_token):
    tokenized = [word_tokenize(sent) for sent in sentence.split()]
    num = 0
    temp_sum = get_vect(".")
    for token in tokenized:
        if num >= max_num_token:
            break
        temp_sum += get_vect(token[0])
        num+=1
    if num == 0:
        return temp_sum
    else:
        return temp_sum/num

In [14]:
!pwd

/workspace/Tugas Akhir/fastText


In [15]:
def get_vector_all_sentence_from_a_column(df, kolom, max_num_token = 256):
    ret = []
    ret1 = [] 
    num = 0
    for row in df['trunc_combination']:
        ret1.append(get_vector_for_sentence(row, max_num_token))
    for row in df[kolom]:
        try:
            ret.append(get_vector_for_sentence(row, max_num_token))
        except:
            ret.append(ret1[num])
        num+=1
    mini=1e10
    for row in ret:
        for sentence in row:
            mini = min(mini, sentence)
    res = []
    for row in ret:
        res.append(row-mini)
    return res

In [16]:
!pip install scikit-learn  -U

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [17]:
import numpy as np
from sklearn.metrics import precision_recall_fscore_support
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
import pandas as pd

def get_eval(ypred, yact):
    p,r, f,s = precision_recall_fscore_support(yact,ypred, average='binary', pos_label=1)
    acc = accuracy_score(yact, ypred)
    print(f"Nilai Precision = {p}")
    print(f"Nilai Recall = {r}")
    print(f"Nilai F1 = {f}")
    return p,r,f

def get_eksperiment_representasi(df_train, df_test, fitur, bencana):
    %cd  ..
    dct_cls = {}
    print(f"Eksperimen pada bencana {bencana} dengan representasi teks {fitur}")
    LABEL2INDEX = { 'N':0,  'Y':1}
    y_train = list(df_train['label'].apply(lambda lab: LABEL2INDEX[lab]))
    y_test = list(df_test['label'].apply(lambda lab: LABEL2INDEX[lab]))
    X_train = get_vector_all_sentence_from_a_column(df_train,fitur)
    X_test = get_vector_all_sentence_from_a_column(df_test,fitur)
    rf = RandomForestClassifier(max_depth=100, n_estimators=100)
    lr = LogisticRegression(solver='liblinear',max_iter = 100)
    mlp = MLPClassifier(random_state=1, max_iter=300)
    xgb = GradientBoostingClassifier(n_estimators=100, learning_rate=0.05,max_depth=3, random_state=0)

    print("XGBOOST")
    xgb.fit(X_train, y_train)
    y_pred_train = xgb.predict(X_train)
    y_pred_test = xgb.predict(X_test)
    print("Di train dapat performa ")
    get_eval(y_pred_train, y_train)
    print("Di test dapat performa")
    p,r,f = get_eval(y_pred_test, y_test)
    dct_metric = {}
    dct_metric["prec"] = p
    dct_metric["rec"] = r
    dct_metric["f1"] = f 
    dct_metric["acc"] = accuracy_score(y_pred_test, y_test)
    dct_cls["xgb"] = dct_metric
    print("########")
    df = df_test[['id']]
    df['label'] = y_pred_test
    df[['id','label']].to_csv(f'../prediction/result_classification_Fasttext/{bencana}_{fitur}_XGBOOST.csv', index=False)

    print("RANDOM FOREST")
    rf.fit(X_train, y_train)
    y_pred_train = rf.predict(X_train)
    y_pred_test = rf.predict(X_test)
    print("Di train dapat performa ")
    get_eval(y_pred_train, y_train)
    print("Di test dapat performa")
    p_l, r_l, f_l = get_eval(y_pred_test, y_test)
    dct_metric = {}
    dct_metric["prec"] = p_l
    dct_metric["rec"] = r_l
    dct_metric["f1"] = f_l 
    dct_metric["acc"] = accuracy_score(y_pred_test, y_test)
    dct_cls["rf"] = dct_metric
    print("########")
    df = df_test[['id']]
    df['label'] = y_pred_test
    df[['id','label']].to_csv(f'../prediction/result_classification_Fasttext/{bencana}_{fitur}_RF.csv', index=False)
    p+=p_l
    r+=r_l
    f+=f_l

    print("LOGISTIK REGRESSION")
    lr.fit(X_train, y_train)
    y_pred_train = lr.predict(X_train)
    y_pred_test = lr.predict(X_test)
    print("Di train dapat performa ")
    get_eval(y_pred_train, y_train)
    print("Di test dapat performa")
    p_l, r_l, f_l = get_eval(y_pred_test, y_test)
    dct_metric = {}
    dct_metric["prec"] = p_l
    dct_metric["rec"] = r_l
    dct_metric["f1"] = f_l 
    dct_metric["acc"] = accuracy_score(y_pred_test, y_test)
    dct_cls["lr"] = dct_metric
    print("########")
    df = df_test[['id']]
    df['label'] = y_pred_test
    df[['id','label']].to_csv(f'../prediction/result_classification_Fasttext/{bencana}_{fitur}_LR.csv', index=False)

    print("MLP")
    mlp.fit(X_train, y_train)
    y_pred_train = mlp.predict(X_train)
    y_pred_test = mlp.predict(X_test)
    print("Di train dapat performa ")
    get_eval(y_pred_train, y_train)
    print("Di test dapat performa")
    p_l, r_l, f_l = get_eval(y_pred_test, y_test)
    dct_metric = {}
    dct_metric["prec"] = p_l
    dct_metric["rec"] = r_l
    dct_metric["f1"] = f_l 
    dct_metric["acc"] = accuracy_score(y_pred_test, y_test)
    dct_cls["mlp"] = dct_metric
    print("########")
    p+=p_l
    r+=r_l
    f+=f_l
    df = df_test[['id']]
    df['label'] = y_pred_test
    df[['id','label']].to_csv(f'../prediction/result_classification_Fasttext/{bencana}_{fitur}_MLP.csv', index=False)

    %cd -
    print('Rata-rata nilai test dengan Classifier xgb, rf, dan mlp adalah')
    print(f'Precision = {p/3}\nRecall = {r/3}\nF1 = {f/3}')
    return dct_cls

def get_f1_eks_representasi_teks_bencana(bencana):
    lst_fitur = ["id","title_ori", "text_ori", "tc_ori", "filter_word_tc","filter_word_tc_window", "trunc_first_short", "trunc_last_short", "trunc_first_long", "trunc_last_long", "trunc_combination",  "textrank", "sum_indobart", "sum_liputan6","sum_pegasus","label"]
    df_train = pd.read_csv(f'{bencana}_train.csv')[lst_fitur]
    df_dev = pd.read_csv(f'{bencana}_dev.csv')[lst_fitur]
    df_test = pd.read_csv(f'{bencana}_test.csv')[lst_fitur]
    print(f"evaluasi pada bencana = {bencana}")
    dct_bencana={}
    dct_bencana[bencana] = {}
    for fitur in lst_fitur[1:-1]:
        dct_fitur = {}
        dct_bencana[bencana][fitur] = get_eksperiment_representasi(df_train, df_test, fitur, bencana)

        print('\n')
    print('************************************************************')


    print(dct_bencana)

In [20]:
%cd ../dataset

/workspace/Tugas Akhir/dataset


In [21]:
import pandas as pd

In [22]:
%cd angin_topan
get_f1_eks_representasi_teks_bencana('angin_topan')
%cd ..

/workspace/Tugas Akhir/dataset/angin_topan
evaluasi pada bencana = angin_topan
/workspace/Tugas Akhir/dataset
Eksperimen pada bencana angin_topan dengan representasi teks title_ori
XGBOOST
Di train dapat performa 
Nilai Precision = 0.9855072463768116
Nilai Recall = 0.8553459119496856
Nilai F1 = 0.9158249158249159
Di test dapat performa
Nilai Precision = 0.0
Nilai Recall = 0.0
Nilai F1 = 0.0
########
RANDOM FOREST


/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/tmp/ipykernel_83597/3013171943.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test


Di train dapat performa 
Nilai Precision = 1.0
Nilai Recall = 1.0
Nilai F1 = 1.0
Di test dapat performa
Nilai Precision = 0.0
Nilai Recall = 0.0
Nilai F1 = 0.0
########
LOGISTIK REGRESSION
Di train dapat performa 
Nilai Precision = 1.0
Nilai Recall = 0.03773584905660377
Nilai F1 = 0.07272727272727272
Di test dapat performa
Nilai Precision = 0.9166666666666666
Nilai Recall = 0.2391304347826087
Nilai F1 = 0.3793103448275862
########
MLP


/tmp/ipykernel_83597/3013171943.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test
/tmp/ipykernel_83597/3013171943.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test
/opt/conda/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/tmp/ipykernel_83597/3013171943.py:112: SettingWithCopyWarning

Di train dapat performa 
Nilai Precision = 0.6601307189542484
Nilai Recall = 0.6352201257861635
Nilai F1 = 0.6474358974358975
Di test dapat performa
Nilai Precision = 0.6818181818181818
Nilai Recall = 0.6521739130434783
Nilai F1 = 0.6666666666666666
########
/workspace/Tugas Akhir/dataset/angin_topan
Rata-rata nilai test dengan Classifier xgb, rf, dan mlp adalah
Precision = 0.22727272727272727
Recall = 0.21739130434782608
F1 = 0.2222222222222222


/workspace/Tugas Akhir/dataset
Eksperimen pada bencana angin_topan dengan representasi teks text_ori
XGBOOST
Di train dapat performa 
Nilai Precision = 1.0
Nilai Recall = 0.9182389937106918
Nilai F1 = 0.9573770491803278
Di test dapat performa
Nilai Precision = 0.375
Nilai Recall = 0.9130434782608695
Nilai F1 = 0.5316455696202531
########
RANDOM FOREST


...

/tmp/ipykernel_83597/3013171943.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test


Di train dapat performa 
Nilai Precision = 1.0
Nilai Recall = 1.0
Nilai F1 = 1.0
Di test dapat performa
Nilai Precision = 0.23
Nilai Recall = 1.0
Nilai F1 = 0.3739837398373984
########
LOGISTIK REGRESSION
Di train dapat performa 
Nilai Precision = 0.0
Nilai Recall = 0.0
Nilai F1 = 0.0
Di test dapat performa
Nilai Precision = 0.6111111111111112
Nilai Recall = 0.2391304347826087
Nilai F1 = 0.34375
########
MLP


/tmp/ipykernel_83597/3013171943.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/tmp/ipykernel_83597/3013171943.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] =

Di train dapat performa 
Nilai Precision = 0.0
Nilai Recall = 0.0
Nilai F1 = 0.0
Di test dapat performa
Nilai Precision = 0.0
Nilai Recall = 0.0
Nilai F1 = 0.0
########
/workspace/Tugas Akhir/dataset/angin_topan
Rata-rata nilai test dengan Classifier xgb, rf, dan mlp adalah
Precision = 0.15333333333333335
Recall = 0.6666666666666666
F1 = 0.24932249322493227


************************************************************
{'angin_topan': {'title_ori': {'xgb': {'prec': 0.0, 'rec': 0.0, 'f1': 0.0, 'acc': 0.77}, 'rf': {'prec': 0.0, 'rec': 0.0, 'f1': 0.0, 'acc': 0.765}, 'lr': {'prec': 0.9166666666666666, 'rec': 0.2391304347826087, 'f1': 0.3793103448275862, 'acc': 0.82}, 'mlp': {'prec': 0.6818181818181818, 'rec': 0.6521739130434783, 'f1': 0.6666666666666666, 'acc': 0.85}}, 'text_ori': {'xgb': {'prec': 0.375, 'rec': 0.9130434782608695, 'f1': 0.5316455696202531, 'acc': 0.63}, 'rf': {'prec': 0.65, 'rec': 0.2826086956521739, 'f1': 0.3939393939393939, 'acc': 0.8}, 'lr': {'prec': 0.0, 'rec': 0.0, 'f

/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/tmp/ipykernel_83597/3013171943.py:112: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test


In [ ]:
%cd banjir
get_f1_eks_representasi_teks_bencana('banjir')
%cd ..

In [20]:
%cd erupsi
get_f1_eks_representasi_teks_bencana('erupsi')
%cd ..

/workspace/dataset/erupsi
evaluasi pada bencana = erupsi
/workspace/dataset
Eksperimen pada bencana erupsi dengan representasi teks title_ori
XGBOOST
Di train dapat performa 
Nilai Precision = 1.0
Nilai Recall = 0.9368421052631579
Nilai F1 = 0.967391304347826
Di test dapat performa
Nilai Precision = 0.38461538461538464
Nilai Recall = 0.14705882352941177
Nilai F1 = 0.21276595744680854
########
RANDOM FOREST


/tmp/ipykernel_1016/527456267.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test


Di train dapat performa 
Nilai Precision = 1.0
Nilai Recall = 1.0
Nilai F1 = 1.0
Di test dapat performa
Nilai Precision = 0.5833333333333334
Nilai Recall = 0.20588235294117646
Nilai F1 = 0.3043478260869565
########
LOGISTIK REGRESSION
Di train dapat performa 
Nilai Precision = 0.0
Nilai Recall = 0.0
Nilai F1 = 0.0
Di test dapat performa
Nilai Precision = 0.0
Nilai Recall = 0.0
Nilai F1 = 0.0
########
MLP


/tmp/ipykernel_1016/527456267.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/tmp/ipykernel_1016/527456267.py:91: Sett

Di train dapat performa 
Nilai Precision = 0.6666666666666666
Nilai Recall = 0.021052631578947368
Nilai F1 = 0.04081632653061225
Di test dapat performa
Nilai Precision = 0.0
Nilai Recall = 0.0
Nilai F1 = 0.0
########
/workspace/dataset/erupsi
Rata-rata nilai test dengan Classifier xgb, rf, dan mlp adalah
Precision = 0.32264957264957267
Recall = 0.1176470588235294
F1 = 0.17237126117792168


/workspace/dataset
Eksperimen pada bencana erupsi dengan representasi teks text_ori


...

/tmp/ipykernel_1016/527456267.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test


Di train dapat performa 
Nilai Precision = 0.0
Nilai Recall = 0.0
Nilai F1 = 0.0
Di test dapat performa
Nilai Precision = 0.0
Nilai Recall = 0.0
Nilai F1 = 0.0
########
MLP


/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/tmp/ipykernel_1016/527456267.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test


Di train dapat performa 
Nilai Precision = 0.0
Nilai Recall = 0.0
Nilai F1 = 0.0
Di test dapat performa
Nilai Precision = 0.0
Nilai Recall = 0.0
Nilai F1 = 0.0
########
/workspace/dataset/erupsi
Rata-rata nilai test dengan Classifier xgb, rf, dan mlp adalah
Precision = 0.11653116531165313
Recall = 0.37254901960784315
F1 = 0.17535418337342956


************************************************************
{'erupsi': {'title_ori': {'xgb': {'prec': 0.38461538461538464, 'rec': 0.14705882352941177, 'f1': 0.21276595744680854, 'acc': 0.815}, 'rf': {'prec': 0.5833333333333334, 'rec': 0.20588235294117646, 'f1': 0.3043478260869565, 'acc': 0.84}, 'lr': {'prec': 0.0, 'rec': 0.0, 'f1': 0.0, 'acc': 0.83}, 'mlp': {'prec': 0.0, 'rec': 0.0, 'f1': 0.0, 'acc': 0.83}}, 'text_ori': {'xgb': {'prec': 1.0, 'rec': 0.20588235294117646, 'f1': 0.34146341463414637, 'acc': 0.865}, 'rf': {'prec': 1.0, 'rec': 0.17647058823529413, 'f1': 0.3, 'acc': 0.86}, 'lr': {'prec': 0.0, 'rec': 0.0, 'f1': 0.0, 'acc': 0.83}, 'mlp': 

/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/tmp/ipykernel_1016/527456267.py:113: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test


In [21]:
%cd gempa
get_f1_eks_representasi_teks_bencana('gempa')
%cd ..

/workspace/dataset/gempa
evaluasi pada bencana = gempa
/workspace/dataset
Eksperimen pada bencana gempa dengan representasi teks title_ori
XGBOOST
Di train dapat performa 
Nilai Precision = 0.9856115107913669
Nilai Recall = 0.7828571428571428
Nilai F1 = 0.8726114649681529
Di test dapat performa
Nilai Precision = 0.22613065326633167
Nilai Recall = 1.0
Nilai F1 = 0.36885245901639346
########
RANDOM FOREST


/tmp/ipykernel_1016/527456267.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test


Di train dapat performa 
Nilai Precision = 1.0
Nilai Recall = 1.0
Nilai F1 = 1.0
Di test dapat performa
Nilai Precision = 0.24074074074074073
Nilai Recall = 0.8666666666666667
Nilai F1 = 0.37681159420289856
########
LOGISTIK REGRESSION


/tmp/ipykernel_1016/527456267.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test


Di train dapat performa 
Nilai Precision = 0.5
Nilai Recall = 0.005714285714285714
Nilai F1 = 0.01129943502824859
Di test dapat performa
Nilai Precision = 0.0
Nilai Recall = 0.0
Nilai F1 = 0.0
########
MLP


...

/tmp/ipykernel_1016/527456267.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test


Di train dapat performa 
Nilai Precision = 1.0
Nilai Recall = 0.005714285714285714
Nilai F1 = 0.011363636363636362
Di test dapat performa
Nilai Precision = 0.0
Nilai Recall = 0.0
Nilai F1 = 0.0
########
MLP


/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/tmp/ipykernel_1016/527456267.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test


Di train dapat performa 
Nilai Precision = 0.0
Nilai Recall = 0.0
Nilai F1 = 0.0
Di test dapat performa
Nilai Precision = 0.0
Nilai Recall = 0.0
Nilai F1 = 0.0
########
/workspace/dataset/gempa
Rata-rata nilai test dengan Classifier xgb, rf, dan mlp adalah
Precision = 0.14423076923076925
Recall = 0.03703703703703704
F1 = 0.05855562784645413


************************************************************
{'gempa': {'title_ori': {'xgb': {'prec': 0.22613065326633167, 'rec': 1.0, 'f1': 0.36885245901639346, 'acc': 0.23}, 'rf': {'prec': 0.24074074074074073, 'rec': 0.8666666666666667, 'f1': 0.37681159420289856, 'acc': 0.355}, 'lr': {'prec': 0.0, 'rec': 0.0, 'f1': 0.0, 'acc': 0.76}, 'mlp': {'prec': 0.0, 'rec': 0.0, 'f1': 0.0, 'acc': 0.765}}, 'text_ori': {'xgb': {'prec': 0.7857142857142857, 'rec': 0.24444444444444444, 'f1': 0.3728813559322034, 'acc': 0.815}, 'rf': {'prec': 0.6666666666666666, 'rec': 0.08888888888888889, 'f1': 0.1568627450980392, 'acc': 0.785}, 'lr': {'prec': 0.0, 'rec': 0.0, 'f1

/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/tmp/ipykernel_1016/527456267.py:113: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test


In [18]:
%cd karhutla
get_f1_eks_representasi_teks_bencana('karhutla')
%cd ..

/workspace/dataset/karhutla
evaluasi pada bencana = karhutla
/workspace/dataset
Eksperimen pada bencana karhutla dengan representasi teks title_ori
XGBOOST
Di train dapat performa 
Nilai Precision = 0.9767441860465116
Nilai Recall = 0.8689655172413793
Nilai F1 = 0.9197080291970803
Di test dapat performa
Nilai Precision = 0.28
Nilai Recall = 1.0
Nilai F1 = 0.43750000000000006
########
RANDOM FOREST


/tmp/ipykernel_2258012/527456267.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test


Di train dapat performa 
Nilai Precision = 1.0
Nilai Recall = 0.996551724137931
Nilai F1 = 0.9982728842832469
Di test dapat performa
Nilai Precision = 0.27918781725888325
Nilai Recall = 0.9821428571428571
Nilai F1 = 0.43478260869565216
########
LOGISTIK REGRESSION


/tmp/ipykernel_2258012/527456267.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test


Di train dapat performa 
Nilai Precision = 0.7986577181208053
Nilai Recall = 0.4103448275862069
Nilai F1 = 0.5421412300683371
Di test dapat performa
Nilai Precision = 0.6956521739130435
Nilai Recall = 0.5714285714285714
Nilai F1 = 0.6274509803921569
########
MLP


...

/tmp/ipykernel_2258012/527456267.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test


Di train dapat performa 
Nilai Precision = 0.7794117647058824
Nilai Recall = 0.18275862068965518
Nilai F1 = 0.29608938547486036
Di test dapat performa
Nilai Precision = 0.7142857142857143
Nilai Recall = 0.17857142857142858
Nilai F1 = 0.2857142857142857
########
MLP


/tmp/ipykernel_2258012/527456267.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test


Di train dapat performa 
Nilai Precision = 0.7721518987341772
Nilai Recall = 0.2103448275862069
Nilai F1 = 0.33062330623306235
Di test dapat performa
Nilai Precision = 0.6086956521739131
Nilai Recall = 0.25
Nilai F1 = 0.35443037974683544
########
/workspace/dataset/karhutla
Rata-rata nilai test dengan Classifier xgb, rf, dan mlp adalah
Precision = 0.5838509316770186
Recall = 0.2976190476190476
F1 = 0.3916477334198853


************************************************************
{'karhutla': {'title_ori': {'xgb': {'prec': 0.28, 'rec': 1.0, 'f1': 0.43750000000000006, 'acc': 0.28}, 'rf': {'prec': 0.27918781725888325, 'rec': 0.9821428571428571, 'f1': 0.43478260869565216, 'acc': 0.285}, 'lr': {'prec': 0.6956521739130435, 'rec': 0.5714285714285714, 'f1': 0.6274509803921569, 'acc': 0.81}, 'mlp': {'prec': 0.46464646464646464, 'rec': 0.8214285714285714, 'f1': 0.5935483870967742, 'acc': 0.685}}, 'text_ori': {'xgb': {'prec': 0.7435897435897436, 'rec': 0.5178571428571429, 'f1': 0.6105263157894737

/tmp/ipykernel_2258012/527456267.py:113: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test


In [ ]:
%cd kekeringan
get_f1_eks_representasi_teks_bencana('kekeringan')
%cd ..

/workspace/dataset/kekeringan
evaluasi pada bencana = kekeringan
/workspace/dataset
Eksperimen pada bencana kekeringan dengan representasi teks title_ori


In [ ]:
%cd longsor
get_f1_eks_representasi_teks_bencana('longsor')
%cd ..

/workspace/dataset/longsor
evaluasi pada bencana = longsor
/workspace/dataset
Eksperimen pada bencana longsor dengan representasi teks title_ori
XGBOOST
Di train dapat performa 
Nilai Precision = 0.9747474747474747
Nilai Recall = 0.8894009216589862
Nilai F1 = 0.9301204819277109
Di test dapat performa
Nilai Precision = 0.5405405405405406
Nilai Recall = 0.37735849056603776
Nilai F1 = 0.4444444444444445
########
RANDOM FOREST


/tmp/ipykernel_1521/527456267.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test


Di train dapat performa 
Nilai Precision = 1.0
Nilai Recall = 1.0
Nilai F1 = 1.0
Di test dapat performa
Nilai Precision = 0.0
Nilai Recall = 0.0
Nilai F1 = 0.0
########
LOGISTIK REGRESSION
Di train dapat performa 
Nilai Precision = 1.0
Nilai Recall = 0.04608294930875576
Nilai F1 = 0.0881057268722467
Di test dapat performa
Nilai Precision = 1.0
Nilai Recall = 0.07547169811320754
Nilai F1 = 0.14035087719298245
########
MLP


/tmp/ipykernel_1521/527456267.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test
/tmp/ipykernel_1521/527456267.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test


Di train dapat performa 
Nilai Precision = 0.7415730337078652
Nilai Recall = 0.6082949308755761
Nilai F1 = 0.6683544303797468
Di test dapat performa
Nilai Precision = 0.7272727272727273
Nilai Recall = 0.4528301886792453
Nilai F1 = 0.5581395348837209
########
/workspace/dataset/longsor
Rata-rata nilai test dengan Classifier xgb, rf, dan mlp adalah
Precision = 0.4226044226044226
Recall = 0.27672955974842767
F1 = 0.33419465977605517


/workspace/dataset
Eksperimen pada bencana longsor dengan representasi teks text_ori


...

/tmp/ipykernel_1521/527456267.py:113: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test


XGBOOST
Di train dapat performa 
Nilai Precision = 0.9851485148514851
Nilai Recall = 0.9170506912442397
Nilai F1 = 0.9498806682577565
Di test dapat performa
Nilai Precision = 0.6065573770491803
Nilai Recall = 0.6981132075471698
Nilai F1 = 0.6491228070175438
########
RANDOM FOREST


/tmp/ipykernel_1521/527456267.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test


Di train dapat performa 
Nilai Precision = 1.0
Nilai Recall = 1.0
Nilai F1 = 1.0
Di test dapat performa
Nilai Precision = 0.8571428571428571
Nilai Recall = 0.11320754716981132
Nilai F1 = 0.2
########
LOGISTIK REGRESSION
Di train dapat performa 
Nilai Precision = 0.5
Nilai Recall = 0.009216589861751152
Nilai F1 = 0.01809954751131222
Di test dapat performa
Nilai Precision = 1.0
Nilai Recall = 0.018867924528301886
Nilai F1 = 0.037037037037037035
########
MLP


/tmp/ipykernel_1521/527456267.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test
/tmp/ipykernel_1521/527456267.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test


In [ ]:
%cd tsunami
get_f1_eks_representasi_teks_bencana('tsunami')
%cd ..